In [5]:
import requests
import os

# Base URL for SoilGrids data
base_url = "https://files.isric.org/soilgrids/latest/data/"
properties = ["bdod", "cec", "clay", "ocs","cfvo","landmask","nitrogen","ocd","phh2o","sand","silt","soc"]  # Properties to download
depths = ["0-5cm", "5-15cm", "15-30cm", "30-60cm"]  # Depth ranges
statistics = ["mean", "Q0.5", "uncertainty"]  # Available statistics

# Define the folder path for saving files
save_folder = os.path.join(os.getcwd(), "soilgrids")
os.makedirs(save_folder, exist_ok=True)  # Create the folder if it doesn't exist

# Function to check if a file exists and download it
def download_file(file_url, save_path):
    head_response = requests.head(file_url)
    if head_response.status_code == 200:
        response = requests.get(file_url, stream=True)
        with open(save_path, 'wb') as file:
            for chunk in response.iter_content(chunk_size=8192):
                file.write(chunk)
        print(f"Downloaded: {save_path}")
    else:
        print(f"File does not exist: {file_url}")

# Loop through properties, depths, and statistics to construct each file's URL and download if available
for prop in properties:
    for depth in depths:
        for stat in statistics:
            file_name = f"{prop}_{depth}_{stat}.vrt"
            file_url = f"{base_url}{prop}/{file_name}"
            save_path = os.path.join(save_folder, file_name)  # Save file in the soilgrids subfolder
            download_file(file_url, save_path)

Downloaded: /Users/maxsonntag/Documents/GitHub/SOC_predictor/soilgrids/bdod_0-5cm_mean.vrt
Downloaded: /Users/maxsonntag/Documents/GitHub/SOC_predictor/soilgrids/bdod_0-5cm_Q0.5.vrt
Downloaded: /Users/maxsonntag/Documents/GitHub/SOC_predictor/soilgrids/bdod_0-5cm_uncertainty.vrt
Downloaded: /Users/maxsonntag/Documents/GitHub/SOC_predictor/soilgrids/bdod_5-15cm_mean.vrt
Downloaded: /Users/maxsonntag/Documents/GitHub/SOC_predictor/soilgrids/bdod_5-15cm_Q0.5.vrt
Downloaded: /Users/maxsonntag/Documents/GitHub/SOC_predictor/soilgrids/bdod_5-15cm_uncertainty.vrt
Downloaded: /Users/maxsonntag/Documents/GitHub/SOC_predictor/soilgrids/bdod_15-30cm_mean.vrt
Downloaded: /Users/maxsonntag/Documents/GitHub/SOC_predictor/soilgrids/bdod_15-30cm_Q0.5.vrt
Downloaded: /Users/maxsonntag/Documents/GitHub/SOC_predictor/soilgrids/bdod_15-30cm_uncertainty.vrt
Downloaded: /Users/maxsonntag/Documents/GitHub/SOC_predictor/soilgrids/bdod_30-60cm_mean.vrt
Downloaded: /Users/maxsonntag/Documents/GitHub/SOC_predict

In [1]:
!pip install requests psycopg2-binary

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 385.8/385.8 kB 4.2 MB/s eta 0:00:00a 0:00:01
  Preparing metadata (setup.py) ... done
  Created wheel for psycopg2-binary: filename=psycopg2_binary-2.9.10-cp312-cp312-macosx_10_15_x86_64.whl size=135498 sha256=c5235b2b20075ca8080742ca99eac767d740287561185ed53ad982c0dab22914
  Stored in directory: /Users/maxsonntag/Library/Caches/pip/wheels/06/bc/a4/bad5bdabd4cf012a00e927db042e0e44d3a649596c548212be
Successfully built psycopg2-binary


In [19]:
import requests

# Example point within Germany
lat, lon = 51.1657, 10.4515
url = f"https://rest.isric.org/soilgrids/v2.0/properties/query?lon={lon}&lat={lat}"

response = requests.get(url)
if response.status_code == 200:
    soil_data = response.json()
    print(soil_data)  # Display the JSON data
else:
    print(f"Error fetching data: {response.status_code}")


{'type': 'Feature', 'geometry': {'type': 'Point', 'coordinates': [10.4515, 51.1657]}, 'properties': {'layers': [{'name': 'bdod', 'unit_measure': {'d_factor': 100, 'mapped_units': 'cg/cm³', 'target_units': 'kg/dm³', 'uncertainty_unit': ''}, 'depths': [{'range': {'top_depth': 0, 'bottom_depth': 5, 'unit_depth': 'cm'}, 'label': '0-5cm', 'values': {'Q0.05': 55, 'Q0.5': 129, 'Q0.95': 167, 'mean': 122, 'uncertainty': 9}}, {'range': {'top_depth': 5, 'bottom_depth': 15, 'unit_depth': 'cm'}, 'label': '5-15cm', 'values': {'Q0.05': 76, 'Q0.5': 136, 'Q0.95': 170, 'mean': 131, 'uncertainty': 7}}, {'range': {'top_depth': 15, 'bottom_depth': 30, 'unit_depth': 'cm'}, 'label': '15-30cm', 'values': {'Q0.05': 113, 'Q0.5': 148, 'Q0.95': 176, 'mean': 147, 'uncertainty': 4}}, {'range': {'top_depth': 30, 'bottom_depth': 60, 'unit_depth': 'cm'}, 'label': '30-60cm', 'values': {'Q0.05': 124, 'Q0.5': 156, 'Q0.95': 179, 'mean': 155, 'uncertainty': 4}}, {'range': {'top_depth': 60, 'bottom_depth': 100, 'unit_depth'

In [21]:
import requests
from requests.auth import HTTPBasicAuth

# Define the WebDAV base URL
base_url = "https://files.isric.org/soilgrids/latest/data/"

# Function to list files in a directory
def list_files(directory_url):
    response = requests.get(directory_url)
    if response.status_code == 200:
        print("Files in directory:")
        print(response.text)
    else:
        print(f"Failed to list directory contents: {response.status_code}")

# List files available in the global data directory
list_files(base_url)

# Function to download a specific file
def download_file(file_url, save_path):
    response = requests.get(file_url, stream=True)
    if response.status_code == 200:
        with open(save_path, 'wb') as file:
            for chunk in response.iter_content(chunk_size=8192):
                file.write(chunk)
        print(f"File downloaded successfully: {save_path}")
    else:
        print(f"Failed to download file: {response.status_code}")

# Example: Download Organic Carbon content at 0-5cm depth
file_url = base_url + "ocs_0-5cm_mean.vrt"
save_path = "ocs_0-5cm_mean.vrt"
download_file(file_url, save_path)


Files in directory:
<!DOCTYPE html>
<html>
<head>
  <meta http-equiv="Content-Type" content="text/html; charset=UTF-8">
  <meta name="generator" content="WsgiDAV/4.2.0">
  <title>ISRIC - Index of /soilgrids/latest/data/ </title>
  <link rel="shortcut icon" href="/:dir_browser/favicon.ico">
  <link rel="stylesheet" href="/:dir_browser/style.css" />
  <script defer src="/:dir_browser/script.js"></script>
  <style type="text/css"> A {behavior: url(#default#AnchorClick);} </style>
</head>

<body onload="onLoad()">

  <h1>
    <img class="logo" alt="ISRIC" title="ISRIC" src="/:dir_browser/logo.png">
    Webdav - Index of /soilgrids/latest/data/
  </h1>
<p>
This page lists all resources made available through ISRIC - World Soil Information's Webdav functionality. 
This URL can also be accessed via the <a href="https://www.isric.org/explore/soilgrids/soilgrids-access">Webdav protocol</a>.
</p><p>
These resources are described in our data catalogue at <a href="https://data.isric.org">data.isri